# 1 下载bert-base-chinese

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from transformers import AutoTokenizer, AutoModel

# 设置模型保存目录
save_directory = "./bert-base-chinese-local"

# 检查模型是否已存在
if os.path.exists(os.path.join(save_directory, "tokenizer_config.json")) and \
   os.path.exists(os.path.join(save_directory, "config.json")):
    print(f"本地模型已存在于 {save_directory}，直接加载")
    tokenizer = AutoTokenizer.from_pretrained(save_directory, local_files_only=True)
    model = AutoModel.from_pretrained(save_directory, local_files_only=True)
    print("模型已从本地成功加载")
else:
    print(f"警告：本地模型目录 {save_directory} 不完整，请确保已正确下载")

/root/miniconda3/envs/multi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


本地模型已存在于 ./bert-base-chinese-local，直接加载
模型已从本地成功加载


# 2

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

# 修改: 添加项目根目录到Python路径 (适应本地环境)
project_root = os.path.dirname(os.path.abspath('__file__'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to system path")

# 打印当前工作目录和Python路径，用于调试
print(f"Current working directory: {os.getcwd()}")
print(f"Python path: {sys.path}")

# 检查src目录是否存在
src_path = os.path.join(project_root, 'src')
if os.path.exists(src_path):
    print(f"src directory found: {src_path}")
else:
    print(f"Warning: src directory not found at {src_path}")

# 显示src/models目录的内容
models_path = os.path.join(src_path, 'models')
if os.path.exists(models_path):
    print(f"Models directory found: {models_path}")
    print("Files in models directory:")
    for f in os.listdir(models_path):
        print(f"  - {f}")
else:
    print(f"Warning: models directory not found at {models_path}")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Added /root/Code/Multi-task Project to system path
Current working directory: /root/Code/Multi-task Project
Python path: ['/root/miniconda3/envs/multi/lib/python310.zip', '/root/miniconda3/envs/multi/lib/python3.10', '/root/miniconda3/envs/multi/lib/python3.10/lib-dynload', '', '/root/miniconda3/envs/multi/lib/python3.10/site-packages', '/tmp/tmpm5exwty3', '/root/Code/Multi-task Project']
src directory found: /root/Code/Multi-task Project/src
Models directory found: /root/Code/Multi-task Project/src/models
Files in models directory:
  - core_seq2seq.py
  - __init__.py
  - __pycache__


# 3 

In [ ]:
import json
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from IPython.display import display, HTML
import sys

# 添加项目根目录到Python路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

# 导入我们自己的Seq2SeqTransformer模型
from src.models.core_seq2seq import Seq2SeqTransformer, PositionalEncoding

# 设置随机种子以保证可重复性
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# 检测设备
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"使用GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU内存总量: {torch.cuda.get_device_properties(0).total_memory / 1024 / 1024 / 1024:.2f} GB")
        print(f"可用GPU数量: {torch.cuda.device_count()}")
    else:
        device = torch.device("cpu")
        print("使用CPU")
    
    return device

# 流式加载JSON数据并随机抽样
def load_json_data_with_sampling(file_path, sample_percentage=10, max_samples=None, task_types=None):
    sample_prob = sample_percentage / 100
    sampled_data = []
    line_count = 0
    
    print(f"Reading file and sampling {sample_percentage}% of data...")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            line_count += 1
            
            if random.random() <= sample_prob:
                try:
                    item = json.loads(line.strip())
                    
                    if task_types is not None and item.get('type') not in task_types:
                        continue
                        
                    sampled_data.append(item)
                    
                    if max_samples and len(sampled_data) >= max_samples:
                        break
                except json.JSONDecodeError:
                    continue

    print(f"Total lines read: {line_count}")
    print(f"Sampled data size: {len(sampled_data)}")
    
    return sampled_data

# 检查标签配置
def fix_label_mapping(data, task_types):
    task_specific_labels = {t: set() for t in task_types}
    
    # 统计每种任务的标签
    for item in data:
        task = item['type']
        if task in task_types and 'answer_choices' in item:
            for choice in item['answer_choices']:
                task_specific_labels[task].add(choice)
            
    print("任务特定标签统计:")
    for task, labels in task_specific_labels.items():
        print(f"{task}: {len(labels)} 个标签 - {list(labels)[:5]}...")

# 自定义数据集类 - 修改以适应多任务模型
class MultiTaskDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512, pad_idx=0):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_idx = pad_idx
        
        # 按任务类型分类数据
        self.task_data = {
            'classify': [],
            'nli': [],
            'mrc': []
        }
        
        for item in data:
            task_type = item['type']
            if task_type in self.task_data:
                self.task_data[task_type].append(item)
            
        # 统计各任务数据量
        self.task_counts = {task: len(items) for task, items in self.task_data.items()}
        print(f"Task data distribution: {self.task_counts}")
        
        # 为分类任务和NLI任务创建标签映射
        self.label_maps = {}
        self._create_label_maps()
        
    def _create_label_maps(self):
        """为每个任务创建标签映射"""
        for task_type in ['classify', 'nli']:
            all_labels = set()
            for item in self.task_data[task_type]:
                if 'answer_choices' in item and item['answer_choices']:
                    for choice in item['answer_choices']:
                        all_labels.add(choice)
                        
            # 为NLI任务标准化标签
            if task_type == 'nli':
                nli_map = {
                    'entailment': 'entailment', '蕴含': 'entailment', '是的': 'entailment', 'yes': 'entailment',
                    'neutral': 'neutral', '中立': 'neutral',
                    'contradiction': 'contradiction', '矛盾': 'contradiction', '不是': 'contradiction', 'no': 'contradiction'
                }
                # 应用映射
                all_labels = set(nli_map.get(label, label) for label in all_labels)
                
            self.label_maps[task_type] = {label: i for i, label in enumerate(sorted(all_labels))}
            print(f"{task_type} task has {len(self.label_maps[task_type])} unique labels")
    
    def __len__(self):
        return sum(self.task_counts.values())
    
    def get_task_item(self, task_type, idx):
        """获取特定任务类型的样本"""
        if idx >= len(self.task_data[task_type]):
            return None
            
        item = self.task_data[task_type][idx]
        
        # 根据任务类型进行不同处理
        if task_type in ['classify', 'nli']:
            return self._process_classification_item(item, task_type)
        elif task_type == 'mrc':
            return self._process_mrc_item(item)
        else:
            return None
    
    def _process_classification_item(self, item, task_type):
        """处理分类和NLI任务的数据项"""
        input_text = item['input']
        target_text = item['target']
        answer_choices = item.get('answer_choices', [])
        
        # 对NLI任务的特殊处理
        if task_type == 'nli':
            nli_map = {
                'entailment': 0, '蕴含': 0, '是的': 0, 'yes': 0,
                'neutral': 1, '中立': 1,
                'contradiction': 2, '矛盾': 2, '不是': 2, 'no': 2
            }
            
            if target_text in nli_map:
                label = torch.tensor(nli_map[target_text], dtype=torch.long)
            elif target_text in answer_choices:
                # 二分类NLI问题(是/否)
                if len(answer_choices) == 2:
                    label = torch.tensor(0 if target_text == answer_choices[0] else 1, dtype=torch.long)
                else:
                    label = torch.tensor(answer_choices.index(target_text), dtype=torch.long)
            else:
                label = torch.tensor(0, dtype=torch.long)  # 默认值
        else:
            # 分类任务处理
            try:
                if target_text in self.label_maps[task_type]:
                    label_idx = self.label_maps[task_type][target_text]
                elif target_text in answer_choices:
                    label_idx = answer_choices.index(target_text)
                else:
                    label_idx = 0
                label = torch.tensor(label_idx, dtype=torch.long)
            except (ValueError, KeyError):
                label = torch.tensor(0, dtype=torch.long)
        
        # 数据增强 (可选)
        if task_type == 'classify' and random.random() < 0.2:
            input_text = self._augment_text(input_text)
            
        # Tokenization
        encoding = self.tokenizer(
            input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt',
            return_token_type_ids=True
        )
        
        # 去掉批次维度
        for key in encoding:
            encoding[key] = encoding[key].squeeze(0)
        
        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'token_type_ids': encoding.get('token_type_ids', None),
            'src_padding_mask': (encoding['input_ids'] == self.pad_idx),
            'label': label,
            'task_type': task_type,
            'target_text': target_text,
            'answer_choices': answer_choices
        }
        
    def _process_mrc_item(self, item):
        """处理MRC任务的数据项 - 改进版"""
        input_text = item['input']
        target_text = item['target'].strip()
        
        # 处理上下文和问题
        context = input_text.split('[SEP]')[0].strip() if '[SEP]' in input_text else input_text
        
        # 使用Transformers的精确偏移映射
        encoding = self.tokenizer(
            input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt',
            return_offsets_mapping=True,
            return_token_type_ids=True
        )
        
        # 提取偏移映射
        offset_mapping = encoding.pop('offset_mapping')[0].numpy()
        
        # 找到答案在上下文中的位置
        start_idx = context.find(target_text)
        start_position = None
        end_position = None
        
        if start_idx != -1:
            end_idx = start_idx + len(target_text)
            
            # 找到对应的token位置
            start_token = None
            end_token = None
            
            for idx, (token_start, token_end) in enumerate(offset_mapping):
                # 找起始token
                if token_start <= start_idx < token_end:
                    start_token = idx
                # 找结束token
                if token_start < end_idx <= token_end and start_token is not None:
                    end_token = idx
                    break
            
            if start_token is not None and end_token is not None:
                start_position = torch.tensor(start_token, dtype=torch.long)
                end_position = torch.tensor(end_token, dtype=torch.long)
            else:
                # 备用方法：寻找包含答案的最小token范围
                answer_tokens = self.tokenizer.encode(
                    target_text, add_special_tokens=False, return_tensors='pt'
                )[0]
                
                input_ids = encoding['input_ids'][0].tolist()
                for i in range(len(input_ids) - len(answer_tokens) + 1):
                    if input_ids[i:i+len(answer_tokens)] == answer_tokens.tolist():
                        start_position = torch.tensor(i, dtype=torch.long)
                        end_position = torch.tensor(i + len(answer_tokens) - 1, dtype=torch.long)
                        break
        
        # 如果仍然找不到，使用默认位置
        if start_position is None or end_position is None:
            # 更合理的默认位置：指向[CLS]标记后的第一个实际token
            start_position = torch.tensor(1, dtype=torch.long)
            end_position = torch.tensor(1, dtype=torch.long)
        
        # 去掉批次维度
        for key in encoding:
            if isinstance(encoding[key], torch.Tensor):
                encoding[key] = encoding[key].squeeze(0)
        
        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'token_type_ids': encoding.get('token_type_ids', None),
            'src_padding_mask': (encoding['input_ids'] == self.pad_idx),
            'start_position': start_position,
            'end_position': end_position,
            'task_type': 'mrc',
            'target_text': target_text
        }
        
    def _augment_text(self, text):
        """文本增强函数"""
        words = list(text)
        # 随机替换、删除或插入字符
        for i in range(min(3, len(words))):
            pos = random.randint(0, len(words) - 1)
            if random.random() < 0.5:  # 替换
                words[pos] = random.choice("的地得了吗呢啊哦嗯呀哈")
            else:  # 删除
                if len(words) > 10:  # 确保不会删除太多
                    words.pop(pos)
        return ''.join(words)
    
    def _augment_mrc(self, input_text, target_text, context):
        """专门为MRC任务设计的数据增强"""
        # 查找上下文中答案的位置
        start_idx = context.find(target_text)
        if start_idx == -1:
            return input_text, target_text
        
        # 如果答案在文本中，有50%的概率保持不变
        if random.random() < 0.5:
            return input_text, target_text
        
        # 对答案进行轻微变换
        aug_operations = [
            # 添加前后缀
            lambda t: t + random.choice(["。", "，", "的"]) if len(t) > 2 else t,
            # 删除末尾字符
            lambda t: t[:-1] if len(t) > 2 else t,
            # 替换一个字符
            lambda t: t if len(t) <= 3 else t[:random.randint(1,len(t)-2)] + random.choice("的一是在") + t[random.randint(1,len(t)-2)+1:]
        ]
        
        # 随机选择一种增强
        aug_op = random.choice(aug_operations)
        new_target = aug_op(target_text)
        
        # 替换原始上下文中的答案
        if new_target != target_text and len(new_target) > 0:
            new_context = context[:start_idx] + new_target + context[start_idx+len(target_text):]
            new_input = new_context
            if '[SEP]' in input_text:
                question = input_text.split('[SEP]')[1]
                new_input = new_context + ' [SEP] ' + question
                
            return new_input, new_target
            
        return input_text, target_text
        
    def __getitem__(self, idx):
        """根据索引获取样本，但按任务类型分配索引"""
        # 计算每个任务的样本量比例
        total = self.__len__()
        task_probs = {task: count/total for task, count in self.task_counts.items()}
        
        # 根据比例随机选择任务类型
        task_type = random.choices(
            list(task_probs.keys()),
            weights=list(task_probs.values()),
            k=1
        )[0]
        
        # 为选中的任务类型随机选择一个索引
        task_idx = random.randint(0, self.task_counts[task_type] - 1)
        
        try:
            return self.get_task_item(task_type, task_idx)
        except Exception as e:
            print(f"Error processing {task_type} item at index {task_idx}: {e}")
            # 返回一个默认项避免批处理失败
            return self.get_task_item('classify', 0) or None

def custom_collate_fn(batch):
    """处理不同大小的批次数据"""
    # 过滤掉None值
    batch = [item for item in batch if item is not None]
    
    if len(batch) == 0:
        return {}
    
    # 首先收集所有样本中出现的键
    all_keys = set()
    for item in batch:
        all_keys.update(item.keys())
    
    result = {}
    for key in all_keys:
        # 只处理在所有样本中都存在的键，或特殊处理某些键
        if all(key in item for item in batch):
            # 对于字符串或列表类型的字段
            if key in ['task_type', 'target_text', 'answer_choices']:
                result[key] = [item[key] for item in batch]
            # 对于张量类型
            elif isinstance(batch[0][key], torch.Tensor):
                shapes = [item[key].shape for item in batch]
                if all(shape == shapes[0] for shape in shapes):
                    # 如果所有形状一致，则进行常规的堆叠
                    result[key] = torch.stack([item[key] for item in batch])
                else:
                    # 如果形状不一致，则进行填充
                    max_len = max(shape[0] for shape in shapes)
                    padded_tensors = []
                    for item in batch:
                        tensor = item[key]
                        if tensor.shape[0] < max_len:
                            padding = torch.zeros(max_len - tensor.shape[0], *tensor.shape[1:], 
                                                dtype=tensor.dtype, device=tensor.device)
                            tensor = torch.cat([tensor, padding], dim=0)
                        padded_tensors.append(tensor)
                    result[key] = torch.stack(padded_tensors)
        else:
            # 特殊处理MRC任务特有的字段
            if key in ['start_position', 'end_position']:
                # 找出拥有该键的样本索引
                indices = [i for i, item in enumerate(batch) if key in item]
                if indices:
                    # 创建包含有效值的列表
                    values = [batch[i][key] for i in indices]
                    # 记录这些样本的索引，以便训练时使用
                    result[f'{key}_indices'] = indices
                    result[key] = torch.stack(values)
    
    return result

# 数据加载器创建函数
def create_dataloaders(train_data, val_data, tokenizer, batch_size=8):
    train_dataset = MultiTaskDataset(train_data, tokenizer)
    val_dataset = MultiTaskDataset(val_data, tokenizer)
    
    # 使用num_workers加速数据加载
    num_workers = min(4, os.cpu_count())
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=num_workers,
        pin_memory=True,  # GPU加速
        collate_fn=custom_collate_fn  # 使用自定义的collate函数
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,  # GPU加速
        collate_fn=custom_collate_fn  # 使用自定义的collate函数
    )
    
    return train_loader, val_loader, train_dataset.label_map

def create_task_specific_dataloaders(train_data, val_data, tokenizer, batch_size=8):
    """创建按任务类型分类的数据加载器"""
    train_dataset = MultiTaskDataset(train_data, tokenizer)
    val_dataset = MultiTaskDataset(val_data, tokenizer)
    
    # 使用num_workers加速数据加载
    num_workers = min(4, os.cpu_count())
    
    # 创建任务特定的数据加载器
    train_loaders = {}
    val_loaders = {}
    
    for task_type in ['classify', 'nli', 'mrc']:
        if train_dataset.task_counts[task_type] > 0:
            # 创建任务特定的数据集包装器
            train_task_dataset = TaskSpecificDataset(train_dataset, task_type)
            train_loaders[task_type] = DataLoader(
                train_task_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=lambda x: task_collate_fn(x, task_type)
            )
        
        if val_dataset.task_counts[task_type] > 0:
            val_task_dataset = TaskSpecificDataset(val_dataset, task_type)
            val_loaders[task_type] = DataLoader(
                val_task_dataset,
                batch_size=batch_size,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=lambda x: task_collate_fn(x, task_type)
            )
    
    return train_loaders, val_loaders, train_dataset.label_maps

class TaskSpecificDataset(Dataset):
    """任务特定的数据集包装器"""
    def __init__(self, multi_task_dataset, task_type):
        self.dataset = multi_task_dataset
        self.task_type = task_type
        self.indices = list(range(self.dataset.task_counts[task_type]))
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        return self.dataset.get_task_item(self.task_type, self.indices[idx])

# 创建任务特定的数据加载器
def task_collate_fn(batch, task_type):
    """任务特定的批处理函数"""
    # 过滤掉None值
    batch = [item for item in batch if item is not None]
    
    if len(batch) == 0:
        return {}
    
    result = {}
    
    # 常规键处理
    for key in batch[0].keys():
        if key in ['task_type', 'target_text', 'answer_choices']:
            result[key] = [item[key] for item in batch]
        elif isinstance(batch[0][key], torch.Tensor):
            result[key] = torch.stack([item[key] for item in batch])
    
    # MRC任务特殊处理
    if task_type == 'mrc':
        if all('start_position' in item and 'end_position' in item for item in batch):
            result['start_position'] = torch.stack([item['start_position'] for item in batch])
            result['end_position'] = torch.stack([item['end_position'] for item in batch])
    
    return result

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
        
    def forward(self, x, target):
        confidence = 1. - self.smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

# 使用预训练的BERT替代自定义编码器
class MultitaskBertModel(nn.Module):
    def __init__(self, num_labels=2):
        super(MultitaskBertModel, self).__init__()
        
        # 加载预训练的BERT模型
        self.bert = AutoModel.from_pretrained("./bert-base-chinese-local")
        
        # 任务特定的头部
        self.classify_head = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.nli_head = nn.Linear(self.bert.config.hidden_size, num_labels)
        
        # 改进MRC头部设计
        hidden_size = self.bert.config.hidden_size
        self.mrc_qa_outputs = nn.Linear(hidden_size, 2)
        
        # 添加MRC专用的双向LSTM层增强上下文理解
        self.mrc_lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size//2,  # 双向所以除2
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )
        
        # 初始化权重
        nn.init.xavier_uniform_(self.classify_head.weight)
        nn.init.zeros_(self.classify_head.bias)

        nn.init.xavier_uniform_(self.nli_head.weight)
        nn.init.zeros_(self.nli_head.bias)

        nn.init.xavier_uniform_(self.mrc_qa_outputs.weight)
        nn.init.zeros_(self.mrc_qa_outputs.bias)
        
        # 冻结BERT底层参数但保持顶层可训练
        for param in list(self.bert.parameters())[:-4]:
            param.requires_grad = False
    
    def _process_mrc(self, sequence_output, attention_mask):
        """处理MRC任务的序列输出"""
        # 首先通过LSTM增强上下文理解
        lstm_output, _ = self.mrc_lstm(sequence_output)
        
        # 然后使用QA输出层获取起始和结束位置
        logits = self.mrc_qa_outputs(lstm_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        
        # 去除多余维度
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        
        # 应用注意力掩码
        if attention_mask is not None:
            # 确保不在有效范围内的位置不被选为答案
            start_logits = start_logits + (1.0 - attention_mask) * -10000.0
            end_logits = end_logits + (1.0 - attention_mask) * -10000.0
        
        return start_logits, end_logits
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, task_type=None):
        # 使用BERT提取特征
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        # 获取序列表示和[CLS]的表示
        sequence_output = outputs.last_hidden_state  # 用于MRC任务
        pooled_output = outputs.pooler_output  # 用于分类任务
        
        if isinstance(task_type, list):
            # 批处理多个样本时使用
            results = []
            mrc_indices = []
            
            for i, t in enumerate(task_type):
                if t == 'classify':
                    results.append(self.classify_head(pooled_output[i:i+1]))
                elif t == 'nli':
                    results.append(self.nli_head(pooled_output[i:i+1]))
                elif t == 'mrc':
                    # 记录MRC样本的索引，稍后单独处理
                    mrc_indices.append(i)
                    # 暂时放入一个占位符
                    dummy = torch.zeros(1, self.classify_head.out_features, device=pooled_output.device)
                    results.append(dummy)
                else:
                    results.append(self.classify_head(pooled_output[i:i+1]))
                    
            # 如果没有MRC样本，直接返回结果
            if not mrc_indices:
                return torch.cat(results, dim=0)
            
            # 处理MRC样本
            mrc_outputs = []
            for i in mrc_indices:
                mrc_input = input_ids[i:i+1]
                mrc_mask = attention_mask[i:i+1] if attention_mask is not None else None
                mrc_token_type = token_type_ids[i:i+1] if token_type_ids is not None else None
                
                start_logits, end_logits = self._process_mrc(
                    sequence_output[i:i+1], 
                    mrc_mask
                )
                
                mrc_outputs.append((start_logits, end_logits))
                
            # 最终返回包括MRC结果的元组
            return torch.cat(results, dim=0), mrc_outputs
        else:
            # 单个样本或同类型批次处理
            if task_type == 'classify':
                return self.classify_head(pooled_output)
            elif task_type == 'nli':
                return self.nli_head(pooled_output)
            elif task_type == 'mrc':
                # 使用改进的MRC处理
                # 首先通过LSTM增强上下文理解
                lstm_output, _ = self.mrc_lstm(sequence_output)
                
                # 然后使用QA输出层获取起始和结束位置
                logits = self.mrc_qa_outputs(lstm_output)
                start_logits, end_logits = logits.split(1, dim=-1)
                
                # 去除多余维度
                start_logits = start_logits.squeeze(-1)
                end_logits = end_logits.squeeze(-1)
                
                # 应用掩码确保只在有效范围内预测
                if attention_mask is not None:
                    start_logits = start_logits + (1 - attention_mask) * -10000.0
                    end_logits = end_logits + (1 - attention_mask) * -10000.0
                
                return start_logits, end_logits
            else:
                return self.classify_head(pooled_output)

# 训练函数 - 优化以适用于GPU
def train(model, train_loader, val_loader, optimizer, scheduler, device, best_model_path, tokenizer, num_epochs=3, eval_steps=100):
    best_val_score = 0
    global_step = 0
    scaler = torch.cuda.amp.GradScaler()  # 使用混合精度训练
    patience = 3  # 早停的耐心参数
    early_stopping_counter = 0  # 早停计数器
    
    # 用于Kaggle输出的结果记录
    results_history = []
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(progress_bar):
            # 检查批次是否为空
            if not batch or len(batch) == 0:
                print("跳过空批次")
                continue
                
            # 检查输入IDs是否存在
            if 'input_ids' not in batch:
                print(f"批次 {batch_idx} 中没有 input_ids，跳过")
                continue
        
        for batch_idx, batch in enumerate(progress_bar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch.get('attention_mask', None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
            src_padding_mask = batch.get('src_padding_mask', None)
            if src_padding_mask is not None:
                src_padding_mask = src_padding_mask.to(device)
            labels = batch['label'].to(device)
            task_types = batch['task_type']
            
            optimizer.zero_grad()
            
            # 仅处理有效样本
            valid_indices = (labels != -100).nonzero(as_tuple=True)[0]
            
            if len(valid_indices) > 0:
                batch_input_ids = input_ids[valid_indices]
                batch_attention_mask = attention_mask[valid_indices] if attention_mask is not None else None
                batch_src_padding_mask = src_padding_mask[valid_indices] if src_padding_mask is not None else None
                batch_labels = labels[valid_indices]
                batch_task_types = [task_types[i] for i in valid_indices]
                
                # 使用混合精度训练
                with torch.cuda.amp.autocast():
                    # 获取token_type_ids并移至设备
                    token_type_ids = batch.get('token_type_ids', None)
                    if token_type_ids is not None:
                        # 先将索引移到与token_type_ids相同的设备上 (CPU)
                        cpu_valid_indices = valid_indices.cpu()
                        # 使用CPU上的索引切片CPU上的张量
                        token_type_ids = token_type_ids[cpu_valid_indices]
                        # 然后将结果移到目标设备
                        token_type_ids = token_type_ids.to(device)
                        
                    outputs = model(
                        input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        token_type_ids=token_type_ids,
                        task_type=batch_task_types
                    )
                    
                    # 分开处理不同任务类型
                    task_weights = {
                        'classify': 1.0,
                        'nli': 2.0,      # 增加NLI任务权重
                        'mrc': 3.0       # 进一步增加MRC任务权重
                    }
                    
                    # 初始化每种任务的损失
                    losses = {}
                    task_counts = {}
                    
                    # 按任务类型分组计算损失
                    for task_type in set(batch_task_types):
                        task_indices = [i for i, t in enumerate(batch_task_types) if t == task_type]
                        task_counts[task_type] = len(task_indices)
                        
                        if task_type in ['classify', 'nli'] and task_indices:
                            task_outputs = outputs[task_indices]
                            task_labels = batch_labels[task_indices]
                            
                            # 使用标签平滑交叉熵来提高泛化能力
                            if task_type == 'nli':
                                # NLI任务可能需要更多正则化
                                loss_fn = LabelSmoothingCrossEntropy(smoothing=0.1)
                            else:
                                loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
                                
                            losses[task_type] = loss_fn(task_outputs, task_labels) * task_weights[task_type]

                    # 在train函数中找到损失计算部分，添加以下代码
                    # 处理MRC任务
                    mrc_indices = [i for i, t in enumerate(batch_task_types) if t == 'mrc']
                    if mrc_indices:
                        # 确保batch中包含start_position和end_position
                        if 'start_position' in batch and 'end_position' in batch:
                            # 获取MRC任务的开始和结束位置
                            start_positions = batch['start_position'].to(device)
                            end_positions = batch['end_position'].to(device)
                            
                            # 只选择MRC类型的样本
                            mrc_start_positions = start_positions[mrc_indices]
                            mrc_end_positions = end_positions[mrc_indices]
                            
                            # 获取MRC任务输入
                            mrc_input_ids = batch_input_ids[mrc_indices]
                            mrc_attention_mask = batch_attention_mask[mrc_indices] if batch_attention_mask is not None else None
                            
                            # 调用模型获取开始和结束位置的logits
                            mrc_start_logits, mrc_end_logits = model(
                                input_ids=mrc_input_ids,
                                attention_mask=mrc_attention_mask,
                                token_type_ids=token_type_ids[mrc_indices] if token_type_ids is not None else None,
                                task_type='mrc'
                            )
                            
                            # 计算MRC损失
                            mrc_start_loss = F.cross_entropy(mrc_start_logits, mrc_start_positions)
                            mrc_end_loss = F.cross_entropy(mrc_end_logits, mrc_end_positions)
                            mrc_loss = (mrc_start_loss + mrc_end_loss) / 2
                            
                            # 添加到损失字典
                            losses['mrc'] = mrc_loss * task_weights['mrc']

                    # 计算总损失
                    if losses:
                        # 根据任务数量计算加权平均损失
                        loss = sum(losses.values()) / len(losses)
                    else:
                        # 如果没有可计算的损失，使用一个小的虚拟损失
                        loss = torch.tensor(0.01, device=device, requires_grad=True)
                
                # 使用混合精度训练优化反向传播
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                
                scheduler.step()
                
                total_loss += loss.item()
                global_step += 1
            
            progress_bar.set_postfix({"Loss": f"{total_loss/(batch_idx+1):.4f}"})
            
            if global_step % eval_steps == 0:                
                val_results = evaluate(model, val_loader, device, tokenizer)
                results_history.append({
                    'step': global_step,
                    'epoch': epoch+1,
                    'val_results': val_results
                })
                
                # 在Kaggle中显示美观的验证结果
                display(HTML(f"""
                <h4>Validation Results at Step {global_step}</h4>
                <ul>
                    <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
                    <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
                    <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
                    <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
                </ul>
                """))
                
                model.train()
                
                val_score = sum(val_results.values()) / len(val_results) if val_results else 0
                if val_score > best_val_score:
                    best_val_score = val_score
                    torch.save(model.state_dict(), best_model_path)
                    print(f"New best model saved with score: {best_val_score:.4f} at {best_model_path}")
        
        # 每个epoch结束后的评估
        val_results = evaluate(model, val_loader, device, tokenizer)
        results_history.append({
            'epoch': epoch+1,
            'val_results': val_results
        })
        
        display(HTML(f"""
        <h3>End of Epoch {epoch+1} Validation Results</h3>
        <ul>
            <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
            <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
            <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
            <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
        </ul>
        """))
        
        val_score = sum(val_results.values()) / len(val_results) if val_results else 0
        if val_score > best_val_score:
            best_val_score = val_score
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with score: {best_val_score:.4f} at {best_model_path}")
            
        # GPU内存回收
        torch.cuda.empty_cache()
    
    return results_history

def train_multitask(model, train_loaders, val_loaders, optimizer, scheduler, device, 
                   best_model_path, tokenizer, num_epochs=3, eval_steps=100):
    """按任务类型分别训练模型"""
    best_val_score = 0
    global_step = 0
    scaler = torch.cuda.amp.GradScaler()
    patience = 3
    early_stopping_counter = 0
    
    # 记录结果历史
    results_history = []
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        batch_count = 0
        
        # 为每个任务创建迭代器
        task_iterators = {task: iter(loader) for task, loader in train_loaders.items()}
        active_tasks = list(task_iterators.keys())
        
        # 估计总批次数
        total_batches = sum(len(loader) for loader in train_loaders.values())
        progress_bar = tqdm(range(total_batches), desc=f"Epoch {epoch+1}/{num_epochs}")
        
        # 任务权重
        task_weights = {
            'classify': 1.0,
            'nli': 1.5,
            'mrc': 5.0
        }
        
        # 按序轮流训练各任务
        while active_tasks:
            for task_type in list(active_tasks):  # 使用list创建副本以便修改
                try:
                    batch = next(task_iterators[task_type])
                    
                    # 处理批次
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch.get('attention_mask', None)
                    if attention_mask is not None:
                        attention_mask = attention_mask.to(device)
                    
                    token_type_ids = batch.get('token_type_ids', None)
                    if token_type_ids is not None:
                        token_type_ids = token_type_ids.to(device)
                    
                    # 针对不同任务类型的处理
                    optimizer.zero_grad()
                    
                    with torch.cuda.amp.autocast():
                        if task_type in ['classify', 'nli']:
                            labels = batch['label'].to(device)
                            outputs = model(
                                input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids,
                                task_type=task_type
                            )
                            
                            # 选择损失函数
                            if task_type == 'nli':
                                loss_fn = LabelSmoothingCrossEntropy(smoothing=0.1)
                            else:
                                loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
                                
                            loss = loss_fn(outputs, labels) * task_weights[task_type]
                            
                        elif task_type == 'mrc':
                            start_positions = batch['start_position'].to(device)
                            end_positions = batch['end_position'].to(device)
                            
                            start_logits, end_logits = model(
                                input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids,
                                task_type='mrc'
                            )
                            
                            # 计算MRC损失
                            start_loss = F.cross_entropy(start_logits, start_positions)
                            end_loss = F.cross_entropy(end_logits, end_positions)
                            loss = (start_loss + end_loss) / 2 * task_weights[task_type]
                    
                    # 优化步骤
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    scaler.step(optimizer)
                    scaler.update()
                    
                    scheduler.step()
                    
                    total_loss += loss.item()
                    batch_count += 1
                    global_step += 1
                    
                    # 更新进度条
                    progress_bar.update(1)
                    progress_bar.set_postfix({"Loss": f"{total_loss/batch_count:.4f}"})
                    
                    # 周期性评估
                    if global_step % eval_steps == 0:
                        val_results = evaluate_multitask(model, val_loaders, device, tokenizer)
                        results_history.append({
                            'step': global_step,
                            'epoch': epoch+1,
                            'val_results': val_results
                        })
                        
                        # 显示结果
                        display(HTML(f"""
                        <h4>Validation Results at Step {global_step}</h4>
                        <ul>
                            <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
                            <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
                            <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
                            <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
                        </ul>
                        """))
                        
                        model.train()
                        
                        # 保存最佳模型
                        val_score = sum(val_results.values()) / len(val_results) if val_results else 0
                        if val_score > best_val_score:
                            best_val_score = val_score
                            torch.save(model.state_dict(), best_model_path)
                            print(f"New best model saved with score: {best_val_score:.4f}")
                            early_stopping_counter = 0
                        else:
                            early_stopping_counter += 1
                            
                except StopIteration:
                    # 当前任务的批次已用完
                    active_tasks.remove(task_type)
        
        # 每个epoch结束后的评估
        val_results = evaluate_multitask(model, val_loaders, device, tokenizer)
        results_history.append({
            'epoch': epoch+1,
            'val_results': val_results
        })
        
        display(HTML(f"""
        <h3>End of Epoch {epoch+1} Validation Results</h3>
        <ul>
            <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
            <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
            <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
            <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
        </ul>
        """))
        
        val_score = sum(val_results.values()) / len(val_results) if val_results else 0
        if val_score > best_val_score:
            best_val_score = val_score
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with score: {best_val_score:.4f}")
        
        # 早停检查
        if early_stopping_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break
            
        # GPU内存回收
        torch.cuda.empty_cache()
    
    return results_history

# 评估函数 - 修改以支持MRC任务
def evaluate(model, dataloader, device, tokenizer):
    model.eval()
    
    all_preds = []
    all_labels = []
    all_task_types = []
    mrc_results = []  # 存储MRC任务的结果
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch.get('attention_mask', None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
            src_padding_mask = batch.get('src_padding_mask', None)
            if src_padding_mask is not None:
                src_padding_mask = src_padding_mask.to(device)
            labels = batch['label'].to(device)
            task_types = batch['task_type']
            target_texts = batch['target_text']
            
            valid_indices = (labels != -100).nonzero(as_tuple=True)[0]
            mrc_indices = [i for i, t in enumerate(task_types) if t == 'mrc']
            
            # 处理分类和NLI任务
            if len(valid_indices) > 0:
                batch_input_ids = input_ids[valid_indices]
                batch_attention_mask = attention_mask[valid_indices] if attention_mask is not None else None
                batch_src_padding_mask = src_padding_mask[valid_indices] if src_padding_mask is not None else None
                batch_labels = labels[valid_indices]
                batch_task_types = [task_types[i] for i in valid_indices]
                
                # 只处理非MRC任务
                non_mrc_indices = [i for i, t in enumerate(batch_task_types) if t != 'mrc']
                if non_mrc_indices:
                    non_mrc_input_ids = batch_input_ids[non_mrc_indices]
                    non_mrc_attention_mask = batch_attention_mask[non_mrc_indices] if batch_attention_mask is not None else None
                    non_mrc_token_type_ids = batch.get('token_type_ids', None)
                    if non_mrc_token_type_ids is not None:
                        cpu_valid_indices = valid_indices.cpu()
                        non_mrc_token_type_ids = non_mrc_token_type_ids[cpu_valid_indices][non_mrc_indices].to(device)
                    
                    non_mrc_task_types = [batch_task_types[i] for i in non_mrc_indices]
                    non_mrc_labels = batch_labels[non_mrc_indices]
                    
                    with torch.cuda.amp.autocast():
                        outputs = model(
                            input_ids=non_mrc_input_ids,
                            attention_mask=non_mrc_attention_mask,
                            token_type_ids=non_mrc_token_type_ids,
                            task_type=non_mrc_task_types
                        )
                    
                    preds = torch.argmax(outputs, dim=-1)
                    preds = preds.cpu().numpy()
                    non_mrc_labels = non_mrc_labels.cpu().numpy()
                    
                    all_preds.extend(preds)
                    all_labels.extend(non_mrc_labels)
                    all_task_types.extend(non_mrc_task_types)
            
            # 单独处理MRC任务
            if mrc_indices:
                for i in mrc_indices:
                    mrc_input_ids = input_ids[i:i+1]
                    mrc_attention_mask = attention_mask[i:i+1] if attention_mask is not None else None
                    mrc_token_type_ids = batch.get('token_type_ids', None)
                    if mrc_token_type_ids is not None:
                        mrc_token_type_ids = mrc_token_type_ids[i:i+1].to(device)
                    
                    with torch.cuda.amp.autocast():
                        start_logits, end_logits = model(
                            input_ids=mrc_input_ids,
                            attention_mask=mrc_attention_mask,
                            token_type_ids=mrc_token_type_ids,
                            task_type='mrc'
                        )
                    
                    # 获取最可能的起始和结束位置
                    start_idx = torch.argmax(start_logits, dim=-1).item()
                    end_idx = torch.argmax(end_logits, dim=-1).item()
                    
                    # 确保end_idx >= start_idx
                    if end_idx < start_idx:
                        end_idx = start_idx
                    
                    # 将token ids转换为文本
                    predicted_answer = tokenizer.decode(mrc_input_ids[0][start_idx:end_idx+1].tolist(),
                                                     skip_special_tokens=True)
                    
                    # 计算精确匹配分数 (EM)
                    target_text = target_texts[i]
                    is_exact_match = predicted_answer.strip() == target_text.strip()
                    
                    mrc_results.append({
                        'predicted': predicted_answer,
                        'target': target_text,
                        'exact_match': is_exact_match
                    })
    
    results = {}
    if all_preds:
        overall_acc = accuracy_score(all_labels, all_preds)
        results['overall_accuracy'] = overall_acc
        
        # 计算F1分数
        f1 = f1_score(all_labels, all_preds, average='weighted')
        results['overall_f1'] = f1
    
    # 计算任务特定指标
    task_metrics = {}
    for task_type in set(all_task_types):
        task_indices = [i for i, t in enumerate(all_task_types) if t == task_type]
        if task_indices:
            task_preds = [all_preds[i] for i in task_indices]
            task_labels = [all_labels[i] for i in task_indices]
            task_acc = accuracy_score(task_labels, task_preds)
            task_f1 = f1_score(task_labels, task_preds, average='weighted')
            
            results[f'{task_type}_accuracy'] = task_acc
            results[f'{task_type}_f1'] = task_f1
            
            task_metrics[task_type] = {
                'accuracy': task_acc,
                'f1': task_f1,
                'support': len(task_indices)
            }
    
    # 计算MRC任务的精确匹配分数
    if mrc_results:
        mrc_em = sum(item['exact_match'] for item in mrc_results) / len(mrc_results)
        results['mrc_em'] = mrc_em
        task_metrics['mrc'] = {
            'exact_match': mrc_em,
            'support': len(mrc_results)
        }
    
    # 在Kaggle中显示详细结果表格
    if task_metrics:
        metrics_df = pd.DataFrame.from_dict(task_metrics, orient='index')
        display(HTML("<h4>Detailed Task Metrics</h4>"))
        display(metrics_df)
    
    return results

def calculate_mrc_metrics(predictions, targets):
    """计算MRC的多种评估指标"""
    exact_matches = 0
    f1_scores = 0
    partial_matches = 0  # 部分重叠
    
    for pred, target in zip(predictions, targets):
        # 精确匹配
        if pred.strip() == target.strip():
            exact_matches += 1
            f1_scores += 1
            partial_matches += 1
            continue
            
        # 计算F1和部分匹配
        if len(pred) > 0:
            # 字符级别F1
            common_chars = set(pred) & set(target)
            if common_chars:
                precision = len(common_chars) / len(pred)
                recall = len(common_chars) / len(target)
                f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                f1_scores += f1
                
                # 部分重叠：如果有一定比例的字符重叠
                if f1 >= 0.5:
                    partial_matches += 1
            
    num_samples = len(predictions)
    if num_samples == 0:
        return 0, 0, 0
    
    return (
        exact_matches / num_samples,  # 精确匹配率
        f1_scores / num_samples,      # 平均F1
        partial_matches / num_samples # 部分匹配率
    )

def evaluate_multitask(model, val_loaders, device, tokenizer):
    """评估多任务模型"""
    model.eval()
    
    task_results = {}
    
    with torch.no_grad():
        for task_type, loader in val_loaders.items():
            if task_type in ['classify', 'nli']:
                all_preds = []
                all_labels = []
                
                for batch in tqdm(loader, desc=f"Evaluating {task_type}"):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch.get('attention_mask', None)
                    if attention_mask is not None:
                        attention_mask = attention_mask.to(device)
                    token_type_ids = batch.get('token_type_ids', None)
                    if token_type_ids is not None:
                        token_type_ids = token_type_ids.to(device)
                    labels = batch['label'].to(device)
                    
                    outputs = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        task_type=task_type
                    )
                    
                    preds = torch.argmax(outputs, dim=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                
                if all_preds:
                    accuracy = accuracy_score(all_labels, all_preds)
                    f1 = f1_score(all_labels, all_preds, average='weighted')
                    task_results[f'{task_type}_accuracy'] = accuracy
                    task_results[f'{task_type}_f1'] = f1
                    
            elif task_type == 'mrc':
                mrc_results = []
                
                for batch in tqdm(loader, desc="Evaluating MRC"):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch.get('attention_mask', None)
                    if attention_mask is not None:
                        attention_mask = attention_mask.to(device)
                    token_type_ids = batch.get('token_type_ids', None)
                    if token_type_ids is not None:
                        token_type_ids = token_type_ids.to(device)
                    target_texts = batch['target_text']
                    
                    start_logits, end_logits = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        task_type='mrc'
                    )
                    
                    # 对每个样本计算预测答案
                    for i in range(input_ids.size(0)):
                        sample_start_logits = start_logits[i]
                        sample_end_logits = end_logits[i]
                        
                        # 获取最可能的起始和结束位置
                        start_idx = torch.argmax(sample_start_logits).item()
                        end_idx = torch.argmax(sample_end_logits).item()
                        
                        # 确保end_idx >= start_idx
                        if end_idx < start_idx:
                            end_idx = start_idx
                            
                        # 限制答案长度
                        if end_idx - start_idx > 30:
                            end_idx = start_idx + 30
                        
                        # 将token ids转换为文本
                        predicted_answer = tokenizer.decode(
                            input_ids[i][start_idx:end_idx+1].tolist(),
                            skip_special_tokens=True
                        )
                        
                        # 计算精确匹配
                        target_text = target_texts[i]
                        is_exact_match = predicted_answer.strip() == target_text.strip()
                        
                        mrc_results.append({
                            'predicted': predicted_answer,
                            'target': target_text,
                            'exact_match': is_exact_match
                        })
                
                # 记录更详细的结果
                if mrc_results:
                    predicted_texts = [item['predicted'] for item in mrc_results]
                    target_texts = [item['target'] for item in mrc_results]
                    
                    # 计算多种指标
                    exact_match, f1, partial_match = calculate_mrc_metrics(predicted_texts, target_texts)
                    
                    # 展示详细诊断信息
                    print(f"\n【MRC诊断信息】")
                    print(f"样本数: {len(mrc_results)}")
                    print(f"精确匹配率: {exact_match:.4f}")
                    print(f"平均F1分数: {f1:.4f}")
                    print(f"部分匹配率: {partial_match:.4f}")
                    
                    # 展示一些样例
                    print("\n【预测样例】")
                    for i in range(min(5, len(mrc_results))):
                        print(f"目标: '{mrc_results[i]['target']}'")
                        print(f"预测: '{mrc_results[i]['predicted']}'")
                        print(f"匹配: {mrc_results[i]['exact_match']}\n")
                    
                    task_results['mrc_em'] = exact_match
                    task_results['mrc_f1'] = f1
                    task_results['mrc_partial'] = partial_match
    
    # 计算总体结果
    if task_results:
        task_results['overall_accuracy'] = sum(
            value for key, value in task_results.items() 
            if key.endswith('_accuracy') or key.endswith('_em')
        ) / sum(1 for key in task_results if key.endswith('_accuracy') or key.endswith('_em'))
    
    # 显示详细结果表格
    metrics_df = pd.DataFrame(
        [[task_results.get(f'{task}_accuracy', None), 
          task_results.get(f'{task}_f1', None), 
          task_results.get(f'{task}_em', None)] 
         for task in ['classify', 'nli', 'mrc']],
        index=['Classify', 'NLI', 'MRC'],
        columns=['Accuracy/EM', 'F1', 'EM']
    )
    display(HTML("<h4>Detailed Task Metrics</h4>"))
    display(metrics_df)
    
    return task_results

def main():
    # 参数设置
    # 检测运行环境并设置相应的路径
    is_kaggle = os.path.exists('/kaggle') and '/kaggle/working' in os.getcwd()

    # 确保使用正确的项目根目录
    project_root = os.path.dirname(os.path.abspath('__file__'))
    possible_paths = [
        os.path.dirname(os.path.abspath('__file__')),  # 当前文件所在目录
        os.getcwd(),  # 当前工作目录
        '/root/Code/Multi-task Project'  # 从错误消息推断的完整路径
    ]

    for path in possible_paths:
        if os.path.exists(path):
            project_root = path
            break

    print(f"Using project root: {project_root}")

    # 定义best_model_path变量
    if is_kaggle:
        model_path = '/kaggle/working/best_multitask_transformer.pt'
        best_model_path = model_path
        output_dir = '/kaggle/working/outputs'
        data_file = '/kaggle/input/multitask-data/data.jsonl'
    else:
        model_paths = [
            os.path.join(project_root, "outputs", "transformer_model", "best_multitask_transformer.pt"),
            os.path.join(project_root, "test_outputs", "multitask_model.pth"),
            os.path.join(project_root, "best_multitask_transformer.pt"),
            "./best_multitask_transformer.pt"
        ]
        
        model_path = None
        for path in model_paths:
            if os.path.exists(path):
                model_path = path
                break
        
        if model_path is None:
            model_path = os.path.join(project_root, "outputs", "transformer_model", "best_multitask_transformer.pt")
            print(f"警告: 没有找到现有模型，将创建一个新模型实例")
        
        best_model_path = model_path
        output_dir = os.path.join(project_root, "outputs", "transformer_model")
        data_file = os.path.join(project_root, "data/processed", "pCLUE_train.json") 

    print(f"使用模型路径: {model_path}")
    
    # 模型训练参数
    sample_percentage = 80
    max_samples = 45000
    batch_size = 16  # 增加批次大小，因为现在每种任务独立处理
    epochs = 8
    lr = 3e-5
    weight_decay = 0.01
    eval_steps = 200
    seed = 42
    task_types = ["classify", "nli", "mrc"]
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 检查数据文件是否存在
    if not os.path.exists(data_file):
        raise FileNotFoundError(f"数据文件 '{data_file}' 不存在，请检查路径")
    
    set_seed(seed)
    device = get_device()
    
    # 显示开始信息
    display(HTML("<h2>Starting Multitask Training with Task-Specific Data Loading</h2>"))
    
    # 加载数据
    all_data = load_json_data_with_sampling(
        data_file,
        sample_percentage=sample_percentage,
        max_samples=max_samples,
        task_types=task_types
    )
    
    # 确保每个样本都有答案选项
    for item in all_data:
        if 'answer_choices' not in item or not item['answer_choices']:
            item['answer_choices'] = ["是的", "不是"] if item['type'] == 'nli' else ["选项A", "选项B"]
    
    # 分割数据集
    train_size = int(0.8 * len(all_data))
    random.shuffle(all_data)
    train_data = all_data[:train_size]
    val_data = all_data[train_size:]
    
    # 加载tokenizer
    local_model_path = "./bert-base-chinese-local"
    if os.path.exists(os.path.join(local_model_path, "tokenizer_config.json")):
        tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)
    else:
        alternative_paths = [
            os.path.join(project_root, "bert-base-chinese-local"),
            "/root/Code/Multi-task Project/bert-base-chinese-local"
        ]
        found_model = False
        for path in alternative_paths:
            if os.path.exists(os.path.join(path, "tokenizer_config.json")):
                tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)
                found_model = True
                break
        
        if not found_model:
            raise ValueError("无法找到本地模型文件，请确认bert-base-chinese-local目录路径")
    
    # 创建任务特定的数据加载器
    train_loaders, val_loaders, label_maps = create_task_specific_dataloaders(
        train_data, val_data, tokenizer, batch_size
    )
    
    # 计算词汇表大小和标签数量
    vocab_size = tokenizer.vocab_size
    num_labels = max(len(label_map) for label_map in label_maps.values()) if label_maps else 2
    print(f"Vocabulary size: {vocab_size}")
    print(f"Max labels for classification: {num_labels}")
    
    # 创建模型
    model = MultitaskBertModel(num_labels=num_labels)
    model.to(device)
    
    # 优化器和学习率调度器设置
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.weight']
    
    task_head_params = {
        'classify': list(model.classify_head.parameters()),
        'nli': list(model.nli_head.parameters()),
        'mrc': list(model.mrc_qa_outputs.parameters()) + list(model.mrc_lstm.parameters())
    }
    
    optimizer_grouped_parameters = [
        # BERT层参数
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and 'bert' in n],
        'weight_decay': weight_decay, 'lr': lr},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay) and 'bert' in n],
        'weight_decay': 0.0, 'lr': lr},
        # 分类任务头参数
        {'params': task_head_params['classify'],
        'weight_decay': weight_decay, 'lr': lr * 10},
        # NLI任务头参数
        {'params': task_head_params['nli'],
        'weight_decay': weight_decay, 'lr': lr * 20},
        # MRC任务头参数
        {'params': task_head_params['mrc'],
        'weight_decay': weight_decay, 'lr': lr * 50}
    ]
    
    optimizer = AdamW(optimizer_grouped_parameters)
    
    # 计算总步数
    total_steps = sum(len(loader) for loader in train_loaders.values()) * epochs
    warmup_steps = total_steps // 10  # 10%的预热步骤
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # 训练模型
    results_history = train_multitask(
        model,
        train_loaders,
        val_loaders,
        optimizer,
        scheduler,
        device,
        best_model_path,
        tokenizer,
        num_epochs=epochs,
        eval_steps=eval_steps
    )
    
    # 最终评估
    if os.path.exists(best_model_path):
        print(f"加载最佳模型用于最终评估: {best_model_path}")
        model.load_state_dict(torch.load(best_model_path))
        model.to(device)
    else:
        print(f"警告：无法找到最佳模型文件，使用当前模型进行评估")
    
    final_results = evaluate_multitask(model, val_loaders, device, tokenizer)
    
    display(HTML("<h2>Final Evaluation Results</h2>"))
    
    # 保存最终配置和结果
    # 提取每个任务的标签映射
    label_maps_inv = {
        task: {v: k for k, v in task_map.items()} 
        for task, task_map in label_maps.items()
    }
    with open(os.path.join(output_dir, "label_maps.json"), "w", encoding="utf-8") as f:
        json.dump(label_maps_inv, f, ensure_ascii=False, indent=2)
    print("Task-specific label maps saved for inference")
    
    # 保存模型配置信息
    config_info = {
        "vocab_size": vocab_size,
        "num_labels": num_labels,
        "task_types": task_types,
        "training_params": {
            "batch_size": batch_size,
            "epochs": epochs,
            "learning_rate": lr
        },
        "best_results": final_results
    }
    
    with open(os.path.join(output_dir, "model_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_info, f, ensure_ascii=False, indent=2)
    print("Model configuration saved")

if __name__ == "__main__":
    main()

# 4